In [1]:
import json
import nltk

### 将文本加载为json格式

In [2]:
input_file = 'tweets.txt'
with open(input_file, 'r') as f:
    lines = f.readlines()
items = [json.loads(x) for x in lines]
tweets = [x['text'] for x in items]

### 定义预处理类
* 大写转小写
* 分词
* 去除标点符号和停用词

In [3]:
class Preprocess:
    def __init__(self):
        self.punctuations = [',',':','_','!','\"','*','>','<','@','~','-','(',')','%','=','\\','^','&','|','#','$','[',']','+',':','#','|'] 
        self.stop_words = set(nltk.corpus.stopwords.words('english'))
    def __call__(self,text,query=False):
        text = text.lower()
        text = nltk.word_tokenize(text)
        text = [x for x in text if x not in self.punctuations and x not in self.stop_words]
        return text

In [4]:
preprocess = Preprocess()
tokens = [preprocess(x) for x in tweets]

In [5]:
print(items[1],end='\n\n')
print(tweets[1], end='\n\n')
print(tokens[1])

{'userName': 'servando', 'clusterNo': 35, 'text': "Mourners recall Sarge Shriver's charity, idealism \n    (AP): AP - R. Sargent Shriver was always an optimist, pio... http://bit.ly/gqMcdG", 'timeStr': 'Sun Jan 23 00:07:48 +0000 2011', 'tweetId': '28967095878287360', 'errorCode': '200', 'textCleaned': ' ', 'relevance': 1}

Mourners recall Sarge Shriver's charity, idealism 
    (AP): AP - R. Sargent Shriver was always an optimist, pio... http://bit.ly/gqMcdG

['mourners', 'recall', 'sarge', 'shriver', "'s", 'charity', 'idealism', 'ap', 'ap', 'r.', 'sargent', 'shriver', 'always', 'optimist', 'pio', '...', 'http', '//bit.ly/gqmcdg']


### 统计词频

In [6]:
dictionary = {}
# 使用集合建立postings，方便去重
for i, token in enumerate(tokens):
    for word in token:
        if word in dictionary:
            dictionary[word].add(i)
        else:
            dictionary[word] = set([i])
# 排序
for k,v in dictionary.items():
    dictionary[k] = sorted(list(dictionary[k]))

### 定义postings merge运算

In [7]:
def union_list(l1, l2):
    result = []
    i = 0
    j = 0
    while (i < len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            result.append(l1[i])
            i += 1
            j += 1
        elif l1[i] < l2[j]:
            result.append(l1[i])
            i += 1
        else:
            result.append(l2[j])
            j += 1
    for x in l1[i:]:
        result.append(x)
    for x in l2[j:]:
        result.append(x)
    return result

def intersection_list(l1, l2):
    result = []
    i = 0
    j = 0
    while (i < len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            result.append(l1[i])
            i += 1
            j += 1
        elif l1[i] < l2[j]:
            i += 1
        else:
            j += 1
    return result

def difference_list(l1,l2):
    result = []
    i = 0
    j = 0
    while (i < len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            i += 1
            j += 1
        elif l1[i] < l2[j]:
            result.append(l1[i])
            i += 1
        else:
            j += 1
    for x in l1[i:]:
        result.append(x)
    return result

### 定义查询运算

In [8]:
ops_rule = {
    '+': 1,
    '*': 2,
    '-': 3,
} ##优先级定义


# 中缀表达式转后缀表达式 https://blog.csdn.net/antineutrino/article/details/6763722
def middle_to_after(ss):
    expression = []
    ops = []
    for item in ss:
        # 遇到运算符时，比较其与S1栈顶运算符的优先级
        if item in ['+', '*', '-']:
            while len(ops) >= 0:
                if len(ops) == 0:
                    ops.append(item)
                    break
                if ops[-1] == '(' or ops_rule[item] > ops_rule[ops[-1]]:
                    ops.append(item)
                    break
                else:
                    expression.append(ops.pop())
                    
        #  如果是左括号“(”，则直接压入S1
        elif item == '(':
            ops.append(item)
            
        # 如果是右括号“)”，则依次弹出S1栈顶的运算符，并压入S2，直到遇到左括号为止，此时将这一对括号丢弃
        elif item == ')':
            while len(ops) > 0:
                op = ops.pop()
                if op == '(':
                    break
                else:
                    expression.append(op)
        
        # 遇到操作数时，将其压入S2
        else:
            expression.append(item)

    while len(ops) > 0:
        expression.append(ops.pop())

    return expression

def expression_to_value(expression):
    stack_value = []
    for item in expression:
        if item in ['+', '*', '-']:
            n2 = stack_value.pop()
            n1 = stack_value.pop()
            result = cal(n1, n2, item)
            stack_value.append(result)
        else:
            stack_value.append(item)
    return stack_value[0]
 
def cal(n1, n2, op):
    if op == '+':
        return union_list(n1,n2)
    if op == '*':
        return intersection_list(n1,n2)
    if op == '-':
        return difference_list(n1,n2)

def query_parse(query):
    tokens = query.split()
    items = []
    i = 0
    for token in tokens:
        if token == 'and':
            items.append('*')
        elif token == 'or':
            items.append('+')
        elif token == 'not':
            items.append('-')
        elif token == '(':
            items.append('(')
        elif token == ')':
            items.append(')')
        else:
            if token in dictionary:
                items.append(dictionary[token])
            else:
                items.append([])
    return items

# 带红色强调字体输出
def toRed( s ):
    return "%c[31;2m%s%c[0m"%('\033', s, '\033')
def print_with_emphasize(line, Q):
    Q = Q.replace('and','')
    Q = Q.replace('and','')
    Q = Q.replace('or','')
    Q = Q.replace('(','')
    Q = Q.replace(')','')
    #只剩关键词语
    line = line.split()
    s_line = Q.split()
    to_be_print = ''
    for l in line:
        if l.lower() in s_line:
            to_be_print += toRed(l) + ' '
        else:
            to_be_print += l + ' '
    print(to_be_print+'\n')

def display_query_result(query):
    temp = query
    query = query.lower().replace(' (', ' ( ').replace(') ', ' ) ')
    results = [tweets[x] for x in expression_to_value(middle_to_after(query_parse(query)))]
    print("查询%s, 得到%d条结果：\n"%(toRed(temp), len(results)))
    for result in results:
        print_with_emphasize(result,query)

# 显示查询交互

In [9]:
import tkinter as tk
from IPython.display import clear_output

In [10]:
win = tk.Tk()
win.title('查询')
win.geometry('700x100')

query = tk.StringVar()
entry = tk.Entry(win, relief='sunken',font='Calibri 23',width=33,textvariable=query)
entry.place(relx=0.43,rely=0.5,anchor='center')

def button_command():
    clear_output()
    query_sentence = query.get()
    display_query_result(query_sentence)

button = tk.Button(win, text='查询', height=2,width=8,command=button_command)
button.place(relx=0.9,rely=0.5,anchor='center')

In [11]:
win.mainloop()

查询weasley or ron not ( Harry not potter ), 得到89条结果：

Bass Angler Magazine ISE Show Video with Dan Wells and Ron Howe ...: Boating Tips · Crank Bait Fishing · Delta t... http://bit.ly/gV3IRD 

Acai Berry as a Medicine and Food | Acai Berry Benefits: by Northcountry BoyPost by Ron Duckett Acai berry is a ... http://bit.ly/i4C4lD 

Chairman Ron Paul announces hearing on the Fed's impact on unemployment, scheduled for next Wednesday at 10 AM Eastern. 

Chairman Ron Paul to Tackle the Fed and Jobs: http://t.co/uWhmOZu via @NewAmericanMag 

Google News #RonPaul Chairman Ron Paul to Tackle the Fed and Jobs - The New American http://goo.gl/fb/CLjEp 

Happy birthday to my boy Ron Weasley 

Happy Bday Ron Weasley 

Ron Paul supporters denounce Rand Paul for Hagel vote http://t.co/6xqoqfY9 

@YUNGRONBOI happy Birthday Ron!!! 

Ron Paul supporters denounce Rand Paul for Hagel #vote - Kentucky Republican Sen. Rand Paul’s... http://t.co/6aNn3GVg #RandPaul #RonPaul 

Ron Weasley's Headed To The Small